In [1]:
!pip install -q -r /workspace/Text2midi/requirements.txt
!pipi install hf_transfer
!pip uninstall torchvision -y
# !pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124

Found existing installation: torchvision 0.23.0+cu128
Uninstalling torchvision-0.23.0+cu128:
  Successfully uninstalled torchvision-0.23.0+cu128


In [ ]:
# Login to HuggingFace
from huggingface_hub import login

try:
    login(token="")
    print("Successfully authenticated with Hugging Face.")
except Exception as e:
    print(f"Authentication failed: {e}")
    print("Please ensure your HF_TOKEN is set correctly in Colab Secrets.")

Successfully authenticated with Hugging Face.


In [4]:
import os
import pickle

os.chdir("/workspace")
vocab_path = "Text2midi/artifacts/vocab_remi.pkl"

with open(vocab_path, "rb") as f:
    remi_tokenizer = pickle.load(f)

print(f"Loaded tokenizer with vocab size {remi_tokenizer.vocab_size}")


Loaded tokenizer with vocab size 524


In [3]:
from datasets import load_dataset
from torch.utils.data import Dataset

gigamidi2 = load_dataset("Metacreation/GigaMIDI", "v2.0.0", split="train")

Loading dataset shards:   0%|          | 0/37 [00:00<?, ?it/s]

### Convert Raw Bytes to MIDI Files

In [14]:
import random

num_samples = 100000

# Generate random indices
random_indices = random.sample(range(len(gigamidi2)), num_samples)

# Select the random samples
sample = gigamidi2.select(random_indices)

print(sample)

Dataset({
    features: ['music', 'NOMML', 'num_tracks', 'TPQN', 'total_notes', 'avg_note_duration', 'avg_velocity', 'min_velocity', 'max_velocity', 'tempo', 'loop_track_idx', 'loop_instrument_type', 'loop_start', 'loop_end', 'loop_duration_beats', 'loop_note_density', 'NOMML_Instrument', 'Type', 'instrument_category: drums-only: 0, all-instruments-with-drums: 1,no-drums: 2', 'md5', 'music_styles_curated', 'music_style_scraped', 'music_style_audio_text_Discogs', 'music_style_audio_text_Lastfm', 'music_style_audio_text_Tagtraum', 'title', 'artist', 'audio_text_matches_score', 'audio_text_matches_sid', 'audio_text_matches_mbid', 'MIDI program number (expressive)', 'instrument_group (expressive)', 'start_tick (expressive)', 'end_tick (expressive)', 'duration_beats (expressive)', 'note_density (expressive)', 'loopability (expressive)'],
    num_rows: 100000
})


In [15]:
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import tqdm

def save_midi_from_bytes(midi_bytes: bytes, output_folder: str, filename: str):
    """
    Save raw MIDI bytes to a .mid file.
    
    Args:
        midi_bytes (bytes): The raw bytes of the MIDI file.
        output_folder (str): Directory where the .mid file will be saved.
        filename (str): Name of the output file (without extension).
    """
    os.makedirs(output_folder, exist_ok=True)  # Ensure folder exists
    file_path = os.path.join(output_folder, f"{filename}.mid")
    
    with open(file_path, "wb") as f:
        f.write(midi_bytes)
    
    # print(f"MIDI file save to: {file_path}")


def save_midi_safe(item):
    try:
        save_midi_from_bytes(item['music'], "Text2midi/gigamidi2_100k", item['md5'])
        return True, item.get('md5', 'unknown')
    except Exception as e:
        return False, item.get('md5', 'unknown'), str(e)

success_count = 0
failures = []

max_workers = 12

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(save_midi_safe, item) for item in sample]
    
    for future in tqdm(futures, desc="Converting MIDI files"):
        result = future.result()
        if result[0]:
            success_count += 1
        else:
            failures.append(result[1:])  # store (md5, error)

print(f"Successfully converted {success_count} files")
if failures:
    print(f"{len(failures)} files failed. Example: {failures[:5]}")

Converting MIDI files:   0%|          | 0/100000 [00:00<?, ?it/s]

Successfully converted 100000 files


### Pretraining Dataset (GigaMIDI)

In [5]:
import os
import glob
import torch
from torch.utils.data import Dataset
from torch.nn import functional as F
import numpy as np

class MidiOnlyDataset(Dataset):
    def __init__(self, midi_folder, remi_tokenizer, max_len=2048, shuffle=False):
        """
        midi_folder: path containing raw MIDI files
        remi_tokenizer: your REMI tokenizer (must support __call__ and lookup)
        max_len: max decoder sequence length
        """
        self.remi_tokenizer = remi_tokenizer
        self.max_len = max_len

        # collect all midi file paths
        self.midi_files = sorted(
            glob.glob(os.path.join(midi_folder, "**/*.mid"), recursive=True)
            + glob.glob(os.path.join(midi_folder, "**/*.midi"), recursive=True)
        )

        if shuffle:
            np.random.shuffle(self.midi_files)

        print("MIDI files found:", len(self.midi_files))

    def __len__(self):
        return len(self.midi_files)

    def __getitem__(self, idx):
        midi_path = self.midi_files[idx]

        # tokenize MIDI
        tokens = self.remi_tokenizer(midi_path)

        if len(tokens.ids) == 0:
            midi_tokens = [
                self.remi_tokenizer["BOS_None"],
                self.remi_tokenizer["EOS_None"]
            ]
        else:
            midi_tokens = (
                [self.remi_tokenizer["BOS_None"]] +
                tokens.ids +
                [self.remi_tokenizer["EOS_None"]]
            )

        # pad/truncate
        if len(midi_tokens) < self.max_len:
            labels = F.pad(
                torch.tensor(midi_tokens, dtype=torch.long),
                (0, self.max_len - len(midi_tokens))
            )
        else:
            labels = torch.tensor(
                midi_tokens[:self.max_len],
                dtype=torch.long
            )

        # returns only labels for LM pretraining
        return labels


### Transformer Components
The below code has been copied, with minor modifications, from https://huggingface.co/amaai-lab/text2midi/blob/main/transformer_model.py. By using the same transformer components as those used by the Text2Midi architecture, we will be able to directly compare the impact of our added CMC compenent.

In [6]:
"""
This code is adapted from https://huggingface.co/amaai-lab/text2midi/blob/main/transformer_model.py.
"""

# from aria.tokenizer import AbsTokenizer
# aria_tokenizer = AbsTokenizer()
import copy
import json
from typing import Optional, Any, Union, Callable
import torch.multiprocessing as mp
from torch.nn import DataParallel
import math
import time
import torch
import os
import warnings
from tqdm import tqdm
from torch import Tensor
# from aria.tokenizer import AbsTokenizer
import pickle
from torch.nn import Module, LayerNorm, Dropout, Linear
from torch.nn.modules.container import ModuleList
from torch.nn.modules.activation import MultiheadAttention
from torch.nn.init import xavier_uniform_
import torch.nn.functional as F
import torch.nn as nn

def _generate_square_subsequent_mask(
        sz: int,
        device: Optional[torch.device] = None,
        dtype: Optional[torch.dtype] = None,
) -> Tensor:
    r"""Generate a square causal mask for the sequence.
    The masked positions are filled with float('-inf'). Unmasked positions are filled with float(0.0).
    """
    if device is None:
        device = torch.device('cpu')
    if dtype is None:
        dtype = torch.float32
    return torch.triu(
        torch.full((sz, sz), float('-inf'), dtype=dtype, device=device),
        diagonal=1,
    )


def _get_seq_len(
        src: Tensor,
        batch_first: bool
) -> Optional[int]:

    if src.is_nested:
        return None
    else:
        src_size = src.size()
        if len(src_size) == 2:
            # unbatched: S, E
            return src_size[0]
        else:
            # batched: B, S, E if batch_first else S, B, E
            seq_len_pos = 1 if batch_first else 0
            return src_size[seq_len_pos]


class PositionalEncoding(nn.Module):
    r"""Inject some information about the relative or absolute position of the tokens in the sequence.
        The positional encodings have the same dimension as the embeddings, so that the two can be summed.
        Here, we use sine and cosine functions of different frequencies.
    .. math:
        \text{PosEncoder}(pos, 2i) = sin(pos/10000^(2i/d_model))
        \text{PosEncoder}(pos, 2i+1) = cos(pos/10000^(2i/d_model))
        \text{where pos is the word position and i is the embed idx)
    Args:
        d_model: the embed dim (required).
        dropout: the dropout value (default=0.1).
        max_len: the max. length of the incoming sequence (default=5000).
    Examples:
        >>> pos_encoder = PositionalEncoding(d_model)
    """

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        # self.register_buffer('pe', pe)
        self.register_parameter('pe', nn.Parameter(pe, requires_grad=False))

    def forward(self, x):
        r"""Inputs of forward function
        Args:
            x: the sequence fed to the positional encoder model (required).
        Shape:
            x: [sequence length, batch size, embed dim]
            output: [sequence length, batch size, embed dim]
        Examples:
            >>> output = pos_encoder(x)
        """
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


class MultiHeadSelfAttention(nn.Module):
    r"""Multi-head self-attention module.
    Args:
        embed_dim (int): The input embedding dimension.
        num_heads (int, optional): The number of attention heads (default: 4).
        dropout (float, optional): The dropout probability (default: 0.1).
        device (torch.device, optional): The device to use (default: None).
        dtype (torch.dtype, optional): The data type to use (default: None).
    Attributes:
        dim_head (int): The dimension of each attention head.
        scale (float): The scaling factor for attention scores.
        heads (int): The number of attention heads.
        to_qkv (nn.Linear): Linear layer for projecting input to query, key, and value.
        to_out (nn.Linear): Linear layer for projecting attention output to the original embedding dimension.
        dropout (nn.Dropout): Dropout layer.
    """

    def __init__(
        self,
        embed_dim: int,
        num_heads: int = 4,
        dropout: float = 0.1,
        batch_first: bool = True,
        device: Optional[torch.device] = None,
        dtype: Optional[torch.dtype] = None,
    ):
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()
        self.embed_dim = embed_dim
        self.batch_first = batch_first
        self.dim_head = embed_dim // num_heads
        self.scale = self.dim_head ** -0.5
        self.heads = num_heads
        hidden_dim = self.dim_head * num_heads
        self.to_qkv = nn.Linear(embed_dim, hidden_dim * 3, bias=False, **factory_kwargs)
        self.to_out = nn.Linear(hidden_dim, embed_dim, bias=False, **factory_kwargs)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor, is_causal: bool = True) -> torch.Tensor:
        
        r"""Forward pass of the multi-head self-attention module.
        Args:
            x (torch.Tensor): The input tensor of shape (batch_size, sequence_length, embed_dim).
        Returns:
            torch.Tensor: The output tensor of shape (batch_size, sequence_length, embed_dim).
        """
        if not self.batch_first:
            x = x.transpose(0, 1)
        b, n, _ = x.size()
        q, k, v = torch.chunk(self.to_qkv(x), chunks=3, dim=-1)
        q, k, v = map(lambda t: t.contiguous().view(b, self.heads, n, -1), (q, k, v))

        out = torch.nn.functional.scaled_dot_product_attention(q, k, v, is_causal=is_causal)
        out = out.contiguous().view(b, n, -1)
        out = self.dropout(out)
        return self.to_out(out)



class TransformerEncoder(Module):
    r"""TransformerEncoder is a stack of N encoder layers.
    Users can build the BERT(https://arxiv.org/abs/1810.04805) model with corresponding parameters.
    Args:
        encoder_layer: an instance of the TransformerEncoderLayer() class (required).
        num_layers: the number of sub-encoder-layers in the encoder (required).
        norm: the layer normalization component (optional).
        enable_nested_tensor: if True, input will automatically convert to nested tensor
            (and convert back on output). This will improve the overall performance of
            TransformerEncoder when padding rate is high. Default: ``True`` (enabled).
    """

    __constants__ = ['norm']

    def __init__(
        self,
        encoder_layer: "TransformerEncoderLayer",
        num_layers: int,
        norm: Optional[Module] = None,
        enable_nested_tensor: bool = True,
        mask_check: bool = True
    ) -> None:
        super().__init__()
        torch._C._log_api_usage_once(f"torch.nn.modules.{self.__class__.__name__}")
        self.layers = _get_clones(encoder_layer, num_layers)
        self.num_layers = num_layers
        self.norm = norm
        # this attribute saves the value providedat object construction
        self.enable_nested_tensor = enable_nested_tensor
        # this attribute controls whether nested tensors are used
        self.use_nested_tensor = enable_nested_tensor
        self.mask_check = mask_check

        enc_layer = "encoder_layer"
        why_not_sparsity_fast_path = ''
        if not isinstance(encoder_layer, torch.nn.TransformerEncoderLayer):
            why_not_sparsity_fast_path = f"{enc_layer} was not TransformerEncoderLayer"
        elif encoder_layer.norm_first :
            why_not_sparsity_fast_path = f"{enc_layer}.norm_first was True"
        elif not encoder_layer.self_attn.batch_first:
            why_not_sparsity_fast_path = (f"{enc_layer}.self_attn.batch_first was not True" +
                                          "(use batch_first for better inference performance)")
        elif not encoder_layer.self_attn._qkv_same_embed_dim:
            why_not_sparsity_fast_path = f"{enc_layer}.self_attn._qkv_same_embed_dim was not True"
        elif encoder_layer.self_attn.in_proj_bias is None:
            why_not_sparsity_fast_path = f"{enc_layer}.self_attn was passed bias=False"
        elif not encoder_layer.activation_relu_or_gelu:
            why_not_sparsity_fast_path = f"{enc_layer}.activation_relu_or_gelu was not True"
        elif not (encoder_layer.norm1.eps == encoder_layer.norm2.eps) :
            why_not_sparsity_fast_path = f"{enc_layer}.norm1.eps was not equal to {enc_layer}.norm2.eps"
        elif encoder_layer.self_attn.num_heads % 2 == 1:
            why_not_sparsity_fast_path = f"{enc_layer}.self_attn.num_heads is odd"

        if enable_nested_tensor and why_not_sparsity_fast_path:
            warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
            self.use_nested_tensor = False


    def forward(
            self,
            src: Tensor,
            mask: Optional[Tensor] = None,
            src_key_padding_mask: Optional[Tensor] = None,
            is_causal: Optional[bool] = None) -> Tensor:
        r"""Pass the input through the encoder layers in turn.
        Args:
            src: the sequence to the encoder (required).
            mask: the mask for the src sequence (optional).
            src_key_padding_mask: the mask for the src keys per batch (optional).
            is_causal: If specified, applies a causal mask as ``mask``.
                Default: ``None``; try to detect a causal mask.
                Warning:
                ``is_causal`` provides a hint that ``mask`` is the
                causal mask. Providing incorrect hints can result in
                incorrect execution, including forward and backward
                compatibility.
        Shape:
            see the docs in :class:`~torch.nn.Transformer`.
        """
        src_key_padding_mask = F._canonical_mask(
            mask=src_key_padding_mask,
            mask_name="src_key_padding_mask",
            other_type=F._none_or_dtype(mask),
            other_name="mask",
            target_type=src.dtype
        )

        mask = F._canonical_mask(
            mask=mask,
            mask_name="mask",
            other_type=None,
            other_name="",
            target_type=src.dtype,
            check_other=False,
        )

        output = src
        convert_to_nested = False
        first_layer = self.layers[0]
        src_key_padding_mask_for_layers = src_key_padding_mask
        why_not_sparsity_fast_path = ''
        str_first_layer = "self.layers[0]"
        batch_first = first_layer.self_attn.batch_first
        # is_fastpath_enabled = torch.backends.mha.get_fastpath_enabled()

        # if not is_fastpath_enabled:
        #     why_not_sparsity_fast_path = "torch.backends.mha.get_fastpath_enabled() was not True"
        if not hasattr(self, "use_nested_tensor"):
            why_not_sparsity_fast_path = "use_nested_tensor attribute not present"
        elif not self.use_nested_tensor:
            why_not_sparsity_fast_path = "self.use_nested_tensor (set in init) was not True"
        elif first_layer.training:
            why_not_sparsity_fast_path = f"{str_first_layer} was in training mode"
        elif not src.dim() == 3:
            why_not_sparsity_fast_path = f"input not batched; expected src.dim() of 3 but got {src.dim()}"
        elif src_key_padding_mask is None:
            why_not_sparsity_fast_path = "src_key_padding_mask was None"
        elif (((not hasattr(self, "mask_check")) or self.mask_check)
                and not torch._nested_tensor_from_mask_left_aligned(src, src_key_padding_mask.logical_not())):
            why_not_sparsity_fast_path = "mask_check enabled, and src and src_key_padding_mask was not left aligned"
        elif output.is_nested:
            why_not_sparsity_fast_path = "NestedTensor input is not supported"
        elif mask is not None:
            why_not_sparsity_fast_path = "src_key_padding_mask and mask were both supplied"
        elif torch.is_autocast_enabled():
            why_not_sparsity_fast_path = "autocast is enabled"

        if not why_not_sparsity_fast_path:
            tensor_args = (
                src,
                first_layer.self_attn.in_proj_weight,
                first_layer.self_attn.in_proj_bias,
                first_layer.self_attn.out_proj.weight,
                first_layer.self_attn.out_proj.bias,
                first_layer.norm1.weight,
                first_layer.norm1.bias,
                first_layer.norm2.weight,
                first_layer.norm2.bias,
                first_layer.linear1.weight,
                first_layer.linear1.bias,
                first_layer.linear2.weight,
                first_layer.linear2.bias,
            )
            _supported_device_type = ["cpu", "cuda", torch.utils.backend_registration._privateuse1_backend_name]
            if torch.overrides.has_torch_function(tensor_args):
                why_not_sparsity_fast_path = "some Tensor argument has_torch_function"
            elif src.device.type not in _supported_device_type:
                why_not_sparsity_fast_path = f"src device is neither one of {_supported_device_type}"
            elif torch.is_grad_enabled() and any(x.requires_grad for x in tensor_args):
                why_not_sparsity_fast_path = ("grad is enabled and at least one of query or the "
                                              "input/output projection weights or biases requires_grad")

            if (not why_not_sparsity_fast_path) and (src_key_padding_mask is not None):
                convert_to_nested = True
                output = torch._nested_tensor_from_mask(output, src_key_padding_mask.logical_not(), mask_check=False)
                src_key_padding_mask_for_layers = None

        seq_len = _get_seq_len(src, batch_first)
        is_causal = _detect_is_causal_mask(mask, is_causal, seq_len)

        for mod in self.layers:
            output = mod(output, src_mask=mask, is_causal=is_causal, src_key_padding_mask=src_key_padding_mask_for_layers)

        if convert_to_nested:
            output = output.to_padded_tensor(0., src.size())

        if self.norm is not None:
            output = self.norm(output)

        return output


class TransformerDecoder(Module):
    r"""TransformerDecoder is a stack of N decoder layers.
    Args:
        decoder_layer: an instance of the TransformerDecoderLayer() class (required).
        num_layers: the number of sub-decoder-layers in the decoder (required).
        norm: the layer normalization component (optional).
    """

    __constants__ = ['norm']

    def __init__(
        self,
        decoder_layer: "TransformerDecoderLayer",
        num_layers: int,
        norm: Optional[Module] = None
    ) -> None:
        super().__init__()
        torch._C._log_api_usage_once(f"torch.nn.modules.{self.__class__.__name__}")
        self.layers = _get_clones(decoder_layer, num_layers)
        self.num_layers = num_layers
        self.norm = norm


    def forward(self, tgt: Tensor, memory: Tensor, tgt_mask: Optional[Tensor] = None,
                memory_mask: Optional[Tensor] = None,
                memory_key_padding_mask: Optional[Tensor] = None, tgt_is_causal: Optional[bool] = None,
                memory_is_causal: bool = False) -> Tensor:
        r"""Pass the inputs (and mask) through the decoder layer in turn.
        Args:
            tgt: the sequence to the decoder (required).
            memory: the sequence from the last layer of the encoder (required).
            tgt_mask: the mask for the tgt sequence (optional).
            memory_mask: the mask for the memory sequence (optional).
            memory_key_padding_mask: the mask for the memory keys per batch (optional).
            tgt_is_causal: If specified, applies a causal mask as ``tgt mask``.
                Default: ``None``; try to detect a causal mask.
                Warning:
                ``tgt_is_causal`` provides a hint that ``tgt_mask`` is
                the causal mask. Providing incorrect hints can result in
                incorrect execution, including forward and backward
                compatibility.
            memory_is_causal: If specified, applies a causal mask as
                ``memory mask``.
                Default: ``False``.
                Warning:
                ``memory_is_causal`` provides a hint that
                ``memory_mask`` is the causal mask. Providing incorrect
                hints can result in incorrect execution, including
                forward and backward compatibility.
        Shape:
            see the docs in :class:`~torch.nn.Transformer`.
        """
        output = tgt

        seq_len = _get_seq_len(tgt, self.layers[0].self_attn.batch_first)
        tgt_is_causal = _detect_is_causal_mask(tgt_mask, tgt_is_causal, seq_len)
        # print(f'target is causal: {tgt_is_causal}')

        for mod in self.layers:
            output = mod(output, memory,
                        memory_mask=memory_mask,                            
                        memory_key_padding_mask=memory_key_padding_mask,
                        tgt_is_causal=tgt_is_causal,
                        memory_is_causal=memory_is_causal)

        if self.norm is not None:
            output = self.norm(output)

        return output



class TransformerEncoderLayer(Module):
    r"""TransformerEncoderLayer is made up of self-attn and feedforward network.
    This standard encoder layer is based on the paper "Attention Is All You Need".
    Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N Gomez,
    Lukasz Kaiser, and Illia Polosukhin. 2017. Attention is all you need. In Advances in
    Neural Information Processing Systems, pages 6000-6010. Users may modify or implement
    in a different way during application.
    TransformerEncoderLayer can handle either traditional torch.tensor inputs,
    or Nested Tensor inputs.  Derived classes are expected to similarly accept
    both input formats.  (Not all combinations of inputs are currently
    supported by TransformerEncoderLayer while Nested Tensor is in prototype
    state.)
    If you are implementing a custom layer, you may derive it either from
    the Module or TransformerEncoderLayer class.  If your custom layer
    supports both torch.Tensors and Nested Tensors inputs, make its
    implementation a derived class of TransformerEncoderLayer. If your custom
    Layer supports only torch.Tensor inputs, derive its implementation from
    Module.
    Args:
        d_model: the number of expected features in the input (required).
        nhead: the number of heads in the multiheadattention models (required).
        dim_feedforward: the dimension of the feedforward network model (default=2048).
        dropout: the dropout value (default=0.1).
        activation: the activation function of the intermediate layer, can be a string
            ("relu" or "gelu") or a unary callable. Default: relu
        layer_norm_eps: the eps value in layer normalization components (default=1e-5).
        batch_first: If ``True``, then the input and output tensors are provided
            as (batch, seq, feature). Default: ``False`` (seq, batch, feature).
        norm_first: if ``True``, layer norm is done prior to attention and feedforward
            operations, respectively. Otherwise it's done after. Default: ``False`` (after).
        bias: If set to ``False``, ``Linear`` and ``LayerNorm`` layers will not learn an additive
            bias. Default: ``True``.
    Examples::
        >>> encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
        >>> src = torch.rand(10, 32, 512)
        >>> out = encoder_layer(src)
    Alternatively, when ``batch_first`` is ``True``:
        >>> encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8, batch_first=True)
        >>> src = torch.rand(32, 10, 512)
        >>> out = encoder_layer(src)
    Fast path:
        forward() will use a special optimized implementation described in
        `FlashAttention: Fast and Memory-Efficient Exact Attention with IO-Awareness`_ if all of the following
        conditions are met:
        - Either autograd is disabled (using ``torch.inference_mode`` or ``torch.no_grad``) or no tensor
          argument ``requires_grad``
        - training is disabled (using ``.eval()``)
        - batch_first is ``True`` and the input is batched (i.e., ``src.dim() == 3``)
        - activation is one of: ``"relu"``, ``"gelu"``, ``torch.functional.relu``, or ``torch.functional.gelu``
        - at most one of ``src_mask`` and ``src_key_padding_mask`` is passed
        - if src is a `NestedTensor <https://pytorch.org/docs/stable/nested.html>`_, neither ``src_mask``
          nor ``src_key_padding_mask`` is passed
        - the two ``LayerNorm`` instances have a consistent ``eps`` value (this will naturally be the case
          unless the caller has manually modified one without modifying the other)
        If the optimized implementation is in use, a
        `NestedTensor <https://pytorch.org/docs/stable/nested.html>`_ can be
        passed for ``src`` to represent padding more efficiently than using a padding
        mask. In this case, a `NestedTensor <https://pytorch.org/docs/stable/nested.html>`_ will be
        returned, and an additional speedup proportional to the fraction of the input that
        is padding can be expected.
        .. _`FlashAttention: Fast and Memory-Efficient Exact Attention with IO-Awareness`:
         https://arxiv.org/abs/2205.14135
    """

    __constants__ = ['norm_first']

    def __init__(self, d_model: int, nhead: int, dim_feedforward: int = 2048, dropout: float = 0.1,
                 activation: Union[str, Callable[[Tensor], Tensor]] = F.relu,
                 layer_norm_eps: float = 1e-5, batch_first: bool = False, norm_first: bool = False,
                 bias: bool = True, device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()
        self.self_attn = MultiheadAttention(d_model, nhead, dropout=dropout,
                                            bias=bias, batch_first=batch_first,
                                            **factory_kwargs)
        # Implementation of Feedforward model
        self.linear1 = Linear(d_model, dim_feedforward, bias=bias, **factory_kwargs)
        self.dropout = Dropout(dropout)
        self.linear2 = Linear(dim_feedforward, d_model, bias=bias, **factory_kwargs)

        self.norm_first = norm_first
        self.norm1 = LayerNorm(d_model, eps=layer_norm_eps, bias=bias, **factory_kwargs)
        self.norm2 = LayerNorm(d_model, eps=layer_norm_eps, bias=bias, **factory_kwargs)
        self.dropout1 = Dropout(dropout)
        self.dropout2 = Dropout(dropout)

        # Legacy string support for activation function.
        if isinstance(activation, str):
            activation = _get_activation_fn(activation)

        # We can't test self.activation in forward() in TorchScript,
        # so stash some information about it instead.
        if activation is F.relu or isinstance(activation, torch.nn.ReLU):
            self.activation_relu_or_gelu = 1
        elif activation is F.gelu or isinstance(activation, torch.nn.GELU):
            self.activation_relu_or_gelu = 2
        else:
            self.activation_relu_or_gelu = 0
        self.activation = activation

    def __setstate__(self, state):
        super().__setstate__(state)
        if not hasattr(self, 'activation'):
            self.activation = F.relu

    def forward(
            self,
            src: Tensor,
            src_mask: Optional[Tensor] = None,
            src_key_padding_mask: Optional[Tensor] = None,
            is_causal: bool = False) -> Tensor:
        r"""Pass the input through the encoder layer.
        Args:
            src: the sequence to the encoder layer (required).
            src_mask: the mask for the src sequence (optional).
            src_key_padding_mask: the mask for the src keys per batch (optional).
            is_causal: If specified, applies a causal mask as ``src mask``.
                Default: ``False``.
                Warning:
                ``is_causal`` provides a hint that ``src_mask`` is the
                causal mask. Providing incorrect hints can result in
                incorrect execution, including forward and backward
                compatibility.
        Shape:
            see the docs in :class:`~torch.nn.Transformer`.
        """
        src_key_padding_mask = F._canonical_mask(
            mask=src_key_padding_mask,
            mask_name="src_key_padding_mask",
            other_type=F._none_or_dtype(src_mask),
            other_name="src_mask",
            target_type=src.dtype
        )

        src_mask = F._canonical_mask(
            mask=src_mask,
            mask_name="src_mask",
            other_type=None,
            other_name="",
            target_type=src.dtype,
            check_other=False,
        )

        # is_fastpath_enabled = torch.backends.mha.get_fastpath_enabled()

        # see Fig. 1 of https://arxiv.org/pdf/2002.04745v1.pdf
        why_not_sparsity_fast_path = ''
        # if not is_fastpath_enabled:
        #     why_not_sparsity_fast_path = "torch.backends.mha.get_fastpath_enabled() was not True"
        if not src.dim() == 3:
            why_not_sparsity_fast_path = f"input not batched; expected src.dim() of 3 but got {src.dim()}"
        elif self.training:
            why_not_sparsity_fast_path = "training is enabled"
        elif not self.self_attn.batch_first:
            why_not_sparsity_fast_path = "self_attn.batch_first was not True"
        elif self.self_attn.in_proj_bias is None:
            why_not_sparsity_fast_path = "self_attn was passed bias=False"
        elif not self.self_attn._qkv_same_embed_dim:
            why_not_sparsity_fast_path = "self_attn._qkv_same_embed_dim was not True"
        elif not self.activation_relu_or_gelu:
            why_not_sparsity_fast_path = "activation_relu_or_gelu was not True"
        elif not (self.norm1.eps == self.norm2.eps):
            why_not_sparsity_fast_path = "norm1.eps is not equal to norm2.eps"
        elif src.is_nested and (src_key_padding_mask is not None or src_mask is not None):
            why_not_sparsity_fast_path = "neither src_key_padding_mask nor src_mask are not supported with NestedTensor input"
        elif self.self_attn.num_heads % 2 == 1:
            why_not_sparsity_fast_path = "num_head is odd"
        elif torch.is_autocast_enabled():
            why_not_sparsity_fast_path = "autocast is enabled"
        if not why_not_sparsity_fast_path:
            tensor_args = (
                src,
                self.self_attn.in_proj_weight,
                self.self_attn.in_proj_bias,
                self.self_attn.out_proj.weight,
                self.self_attn.out_proj.bias,
                self.norm1.weight,
                self.norm1.bias,
                self.norm2.weight,
                self.norm2.bias,
                self.linear1.weight,
                self.linear1.bias,
                self.linear2.weight,
                self.linear2.bias,
            )

            # We have to use list comprehensions below because TorchScript does not support
            # generator expressions.
            _supported_device_type = ["cpu", "cuda", torch.utils.backend_registration._privateuse1_backend_name]
            if torch.overrides.has_torch_function(tensor_args):
                why_not_sparsity_fast_path = "some Tensor argument has_torch_function"
            elif not all((x.device.type in _supported_device_type) for x in tensor_args):
                why_not_sparsity_fast_path = ("some Tensor argument's device is neither one of "
                                              f"{_supported_device_type}")
            elif torch.is_grad_enabled() and any(x.requires_grad for x in tensor_args):
                why_not_sparsity_fast_path = ("grad is enabled and at least one of query or the "
                                              "input/output projection weights or biases requires_grad")

            if not why_not_sparsity_fast_path:
                merged_mask, mask_type = self.self_attn.merge_masks(src_mask, src_key_padding_mask, src)
                return torch._transformer_encoder_layer_fwd(
                    src,
                    self.self_attn.embed_dim,
                    self.self_attn.num_heads,
                    self.self_attn.in_proj_weight,
                    self.self_attn.in_proj_bias,
                    self.self_attn.out_proj.weight,
                    self.self_attn.out_proj.bias,
                    self.activation_relu_or_gelu == 2,
                    self.norm_first,
                    self.norm1.eps,
                    self.norm1.weight,
                    self.norm1.bias,
                    self.norm2.weight,
                    self.norm2.bias,
                    self.linear1.weight,
                    self.linear1.bias,
                    self.linear2.weight,
                    self.linear2.bias,
                    merged_mask,
                    mask_type,
                )


        x = src
        if self.norm_first:
            x = x + self._sa_block(self.norm1(x), src_mask, src_key_padding_mask, is_causal=is_causal)
            x = x + self._ff_block(self.norm2(x))
        else:
            x = self.norm1(x + self._sa_block(x, src_mask, src_key_padding_mask, is_causal=is_causal))
            x = self.norm2(x + self._ff_block(x))

        return x


    # self-attention block
    def _sa_block(self, x: Tensor,
                  attn_mask: Optional[Tensor], key_padding_mask: Optional[Tensor], is_causal: bool = False) -> Tensor:
        x = self.self_attn(x, x, x,
                           attn_mask=attn_mask,
                           key_padding_mask=key_padding_mask,
                           need_weights=False, is_causal=is_causal)[0]
        return self.dropout1(x)

    # feed forward block
    def _ff_block(self, x: Tensor) -> Tensor:
        x = self.linear2(self.dropout(self.activation(self.linear1(x))))
        return self.dropout2(x)



class TransformerDecoderLayer(Module):
    r"""TransformerDecoderLayer is made up of self-attn, multi-head-attn and feedforward network.
    This standard decoder layer is based on the paper "Attention Is All You Need".
    Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N Gomez,
    Lukasz Kaiser, and Illia Polosukhin. 2017. Attention is all you need. In Advances in
    Neural Information Processing Systems, pages 6000-6010. Users may modify or implement
    in a different way during application.
    Args:
        d_model: the number of expected features in the input (required).
        nhead: the number of heads in the multiheadattention models (required).
        dim_feedforward: the dimension of the feedforward network model (default=2048).
        dropout: the dropout value (default=0.1).
        activation: the activation function of the intermediate layer, can be a string
            ("relu" or "gelu") or a unary callable. Default: relu
        layer_norm_eps: the eps value in layer normalization components (default=1e-5).
        batch_first: If ``True``, then the input and output tensors are provided
            as (batch, seq, feature). Default: ``False`` (seq, batch, feature).
        norm_first: if ``True``, layer norm is done prior to self attention, multihead
            attention and feedforward operations, respectively. Otherwise it's done after.
            Default: ``False`` (after).
        bias: If set to ``False``, ``Linear`` and ``LayerNorm`` layers will not learn an additive
            bias. Default: ``True``.
    Examples::
        >>> decoder_layer = nn.TransformerDecoderLayer(d_model=512, nhead=8)
        >>> memory = torch.rand(10, 32, 512)
        >>> tgt = torch.rand(20, 32, 512)
        >>> out = decoder_layer(tgt, memory)
    Alternatively, when ``batch_first`` is ``True``:
        >>> decoder_layer = nn.TransformerDecoderLayer(d_model=512, nhead=8, batch_first=True)
        >>> memory = torch.rand(32, 10, 512)
        >>> tgt = torch.rand(32, 20, 512)
        >>> out = decoder_layer(tgt, memory)
    """

    __constants__ = ['norm_first']

    def __init__(self, d_model: int, nhead: int, dim_feedforward: int = 2048,
                 dropout: float = 0.1, activation: Union[str, Callable[[Tensor], Tensor]] = F.relu,
                 layer_norm_eps: float = 1e-5, batch_first: bool = False, norm_first: bool = False,
                 bias: bool = True, device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()

        self.self_attn = MultiHeadSelfAttention(d_model, nhead, dropout=dropout, batch_first=batch_first, **factory_kwargs) 
        self.multihead_attn = MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=batch_first,
                                                 bias=bias, **factory_kwargs)

        # Implementation of Feedforward model
        self.linear1 = Linear(d_model, dim_feedforward, bias=bias, **factory_kwargs)
        self.dropout = Dropout(dropout)
        self.linear2 = Linear(dim_feedforward, d_model, bias=bias, **factory_kwargs)

        self.norm_first = norm_first
        self.norm1 = LayerNorm(d_model, eps=layer_norm_eps, bias=bias, **factory_kwargs)
        self.norm2 = LayerNorm(d_model, eps=layer_norm_eps, bias=bias, **factory_kwargs)
        self.norm3 = LayerNorm(d_model, eps=layer_norm_eps, bias=bias, **factory_kwargs)
        self.dropout1 = Dropout(dropout)
        self.dropout2 = Dropout(dropout)
        self.dropout3 = Dropout(dropout)

        # Legacy string support for activation function.
        if isinstance(activation, str):
            self.activation = _get_activation_fn(activation)
        else:
            self.activation = activation

    def __setstate__(self, state):
        if 'activation' not in state:
            state['activation'] = F.relu
        super().__setstate__(state)


    def forward(
        self,
        tgt: Tensor,
        memory: Tensor,
        memory_mask: Optional[Tensor] = None,
        memory_key_padding_mask: Optional[Tensor] = None,
        tgt_is_causal: bool = False,
        memory_is_causal: bool = False,
    ) -> Tensor:
        r"""Pass the inputs (and mask) through the decoder layer.
        Args:
            tgt: the sequence to the decoder layer (required).
            memory: the sequence from the last layer of the encoder (required).
            memory_mask: the mask for the memory sequence (optional).
            memory_key_padding_mask: the mask for the memory keys per batch (optional).
            tgt_is_causal: If specified, applies a causal mask as ``tgt mask``.
                Default: ``False``.
                Warning:
                ``tgt_is_causal`` provides a hint that ``tgt_mask`` is
                the causal mask. Providing incorrect hints can result in
                incorrect execution, including forward and backward
                compatibility.
            memory_is_causal: If specified, applies a causal mask as
                ``memory mask``.
                Default: ``False``.
                Warning:
                ``memory_is_causal`` provides a hint that
                ``memory_mask`` is the causal mask. Providing incorrect
                hints can result in incorrect execution, including
                forward and backward compatibility.
        Shape:
            see the docs in :class:`~torch.nn.Transformer`.
        """
        # see Fig. 1 of https://arxiv.org/pdf/2002.04745v1.pdf

        x = tgt
        # print(f'target is causal: {tgt_is_causal}')
        if self.norm_first:
            x = x + self._sa_block(self.norm1(x), tgt_is_causal)
            x = x + self._mha_block(self.norm2(x), memory, memory_mask, memory_key_padding_mask, memory_is_causal)
            x = x + self._ff_block(self.norm3(x))
        else:
            x = self.norm1(x + self._sa_block(x, tgt_is_causal))
            x = self.norm2(x + self._mha_block(x, memory, memory_mask, memory_key_padding_mask, memory_is_causal))
            x = self.norm3(x + self._ff_block(x))

        return x


    # self-attention block
    def _sa_block(self, x: Tensor,
                  is_causal: bool = False) -> Tensor:
        x = self.self_attn(x, is_causal=is_causal)
        return self.dropout1(x)

    # multihead attention block
    def _mha_block(self, x: Tensor, mem: Tensor,
                   attn_mask: Optional[Tensor], key_padding_mask: Optional[Tensor], is_causal: bool = False) -> Tensor:
        x = self.multihead_attn(x, mem, mem,
                                attn_mask=attn_mask,
                                key_padding_mask=key_padding_mask,
                                is_causal=is_causal,
                                need_weights=False)[0]
        return self.dropout2(x)

    # feed forward block
    def _ff_block(self, x: Tensor) -> Tensor:
        x = self.linear2(self.dropout(self.activation(self.linear1(x))))
        return self.dropout3(x)



def _get_clones(module, N):
    # FIXME: copy.deepcopy() is not defined on nn.module
    return ModuleList([copy.deepcopy(module) for i in range(N)])


def _get_activation_fn(activation: str) -> Callable[[Tensor], Tensor]:
    if activation == "relu":
        return F.relu
    elif activation == "gelu":
        return F.gelu

    raise RuntimeError(f"activation should be relu/gelu, not {activation}")


def _detect_is_causal_mask(
        mask: Optional[Tensor],
        is_causal: Optional[bool] = None,
        size: Optional[int] = None,
) -> bool:
    """Return whether the given attention mask is causal.
    Warning:
    If ``is_causal`` is not ``None``, its value will be returned as is.  If a
    user supplies an incorrect ``is_causal`` hint,
    ``is_causal=False`` when the mask is in fact a causal attention.mask
       may lead to reduced performance relative to what would be achievable
       with ``is_causal=True``;
    ``is_causal=True`` when the mask is in fact not a causal attention.mask
       may lead to incorrect and unpredictable execution - in some scenarios,
       a causal mask may be applied based on the hint, in other execution
       scenarios the specified mask may be used.  The choice may not appear
       to be deterministic, in that a number of factors like alignment,
       hardware SKU, etc influence the decision whether to use a mask or
       rely on the hint.
    ``size`` if not None, check whether the mask is a causal mask of the provided size
       Otherwise, checks for any causal mask.
    """
    # Prevent type refinement
    make_causal = (is_causal is True)

    if is_causal is None and mask is not None:
        sz = size if size is not None else mask.size(-2)
        causal_comparison = _generate_square_subsequent_mask(
            sz, device=mask.device, dtype=mask.dtype)

        # Do not use `torch.equal` so we handle batched masks by
        # broadcasting the comparison.
        if mask.size() == causal_comparison.size():
            make_causal = bool((mask == causal_comparison).all())
        else:
            make_causal = False

    return make_causal

def check_instruments(genereated_seq):
    ins_present = []
    ins_count = 0
    instrument_list = ["piano", "chromatic", "organ", "guitar", "bass", "strings", "ensemble", "brass", "reed", "drum", "pipe", "synth_lead", "synth_pad", "synth_effect", "ethnic", "percussive", "sfx"]
    for token in genereated_seq:
        try:
            ins, pitch, vel = token
            # print(str(ins))
        except ValueError:
            try:
                ins, pitch = token
            except ValueError:
                ins = token
        if str(ins) in instrument_list:
            # print('coming here')
            
            if ('prefix', 'instrument', str(ins)) not in ins_present and ins_count < 15:
                ins_count += 1
                print(f'adding instrument {ins}')
                ins_present.append(('prefix', 'instrument', str(ins)))
    if ins_present != []:
        genereated_seq = ins_present + ['<S>']+ genereated_seq +['<E>']
    else:
        genereated_seq = genereated_seq +['<E>']
    print(genereated_seq)
    return genereated_seq 

def process_caption(gpu_id, captions, model, tokenizer, r_tokenizer):
    device = gpu_id
    torch.cuda.set_device(gpu_id)
    model.to(gpu_id)
    model.eval()
    for caption in captions:
        src = caption['caption']
        location = caption['location']
        #src = "A cinematic electronic soundtrack that evokes an epic and dark atmosphere, featuring cello, contrabass, and drums. The song is set in A minor with a moderate tempo and a 4/4 time signature, creating an emotional and action-packed ambiance suitable for film."
        '''
        example 1: "A cheerful and melodic pop Christmas song featuring piano, acoustic guitar, vibraphone, bass, and drums, set in the key of Eb minor with a fast tempo of 123 bpm and a 4/4 time signature, creating a joyful and relaxing atmosphere."lmd_full/1/1b9f5f325c2080d345d877f590aa3dbe.mid
        example 2: "A melodic electronic song with ambient elements, featuring piano, acoustic guitar, alto saxophone, string ensemble, and electric bass. Set in G minor with a 4/4 time signature, it moves at a lively Presto tempo. The composition evokes a blend of relaxation and darkness, with hints of happiness and a meditative quality."lmd_full/1/152891ac63017b234c33e75e4a4a28c5.mid
        example 3: "This motivational electronic and pop song features a clean electric guitar, rock organ, synth voice, acoustic guitar, and vibraphone, creating a melodic and uplifting atmosphere. Set in the key of G# minor with a 4/4 time signature, the track moves at an energetic Allegro tempo of 120 beats per minute. The chord progression of Bbm7 and F# adds to the song's inspiring and corporate feel." lmd_full/1/14347e50e9e8149a9da09f49b188180b.mid
        example 4: "This short electronic song in C minor features a brass section, string ensemble, tenor saxophone, clean electric guitar, and slap bass, creating a melodic and slightly dark atmosphere. With a tempo of 124 BPM (Allegro) and a 4/4 time signature, the track incorporates a chord progression of C7/E, Eb6, and Bbm6, adding a touch of corporate and motivational vibes to the overall composition." lmd_full/1/1dc4cd50a5509d8042d27d80bc7e668e.mid
        example 5: "An energetic and melodic electronic trance track with a space and retro vibe, featuring drums, distortion guitar, flute, synth bass, and slap bass. Set in A minor with a fast tempo of 138 BPM, the song maintains a 4/4 time signature throughout its duration." lmd_full/3/3328b854ebe7a2fc9a746ede74c410ae.mid  
        example 6: "A short but energetic rock fragment in C minor, featuring overdriven guitars, electric bass, and drums, with a vivacious tempo of 155 BPM and a 4/4 time signature, evoking a blend of dark and melodic tones." lmd_full/4/4c2232688c5f869b8470a408d197f5e3.mid 
        example 7: "A classical piece with a cinematic flair, this composition is characterized by its fast tempo and 4/4 time signature. The soprano saxophone and flute take turns leading the melody, supported by the lush tones of the string ensemble, acoustic bass, and pan flute. Set in the key of F minor, the harmonic landscape is painted with the chords Gm7b5, Cm7b5, Fm7, Eaug, and Ab/Eb. The overall mood evokes images of film, with hints of Christmas, drama, documentary, and adventure." lmd_full/9/95bce1b489a11829b4fef39200291f60.mid 
        exmaple 8: "A slow, dark, and emotional classical piece featuring cello, violin, and viola, likely to be used in a dramatic film soundtrack. The composition is in the key of C minor with a 4/4 time signature, and the main chord progression consists of Cm, G, Cm, and Fm." lmd_full/a/a22aad98ecfe4b3d8a353c2a72132834.mid
        example 9: "A slow and emotional classical piece, likely used in a film soundtrack, featuring a church organ as the sole instrument. Written in the key of Eb major with a 3/4 time signature, it evokes a sense of drama and romance. The chord progression of Bb7, Eb, and Ab contributes to the relaxing atmosphere throughout the song." lmd_full/a/af4302a036c9df71e0435df9b08f8c4b.mid
        example 10: "A cinematic electronic soundtrack that evokes an epic and dark atmosphere, featuring cello, contrabass, and drums. The song is set in A minor with a moderate tempo and a 4/4 time signature, creating an emotional and action-packed ambiance suitable for film." lmd_full/d/d920b6f451d7a72ae06f154e7c06c4c1.mid
        '''
        inputs = tokenizer(src, return_tensors='pt', padding=True, truncation=True)
        input_ids = nn.utils.rnn.pad_sequence(inputs.input_ids, batch_first=True, padding_value=0)
        input_ids = input_ids.to(device)
        attention_mask =nn.utils.rnn.pad_sequence(inputs.attention_mask, batch_first=True, padding_value=0) 
        attention_mask = attention_mask.to(device)
        output = model.generate(input_ids, attention_mask,max_len=1000,temperature = 0.9)
        output_list = output[0].tolist()
        print(type(output_list))
        # generated_sequences = [dict_tokenizer[token] for token in output_list[0]]
        # generated_sequences = check_instruments(generated_sequences)
        # # generated_sequences = [('prefix', 'instrument', 'bass'), ('prefix', 'instrument', 'guitar'), ('prefix', 'instrument', 'piano'), ('prefix', 'instrument', 'guitar'), '<S>' ]+ generated_sequences +['<E>']
        # generated_sequences = [token for token in generated_sequences]# if token not in ["<SS>", "<S>", "<E>", "<SEP>"]]
        # # print("Generated sequences:", generated_sequences)
        # with open('../../generated_seq.pkl', 'wb') as f:
        #     pickle.dump(generated_sequences, f)
        # mid_dict = aria_tokenizer.detokenize(generated_sequences)
        # mid = mid_dict.to_midi()
        generated_midi = r_tokenizer.decode(output_list)
        # print(type(generated_midi))
        generated_midi.dump_midi(f"../res/{location}")

def test_generate():
    device = 'cuda'
    artifact_folder = '../artifacts'
    tokenizer_filepath = os.path.join(artifact_folder, "vocab_remi.pkl")
    caption_dataset_path = '/root/captions/train.json'
    print(f'caption_dataset_path: {caption_dataset_path}')
# Load the tokenizer dictionary
    with open(tokenizer_filepath, "rb") as f:
        r_tokenizer = pickle.load(f)
    vocab_size = len(r_tokenizer)#+1
    print("Vocab size: ", vocab_size)
    # print(tokenizer[2171])
    # d_model =
    # model = Transformer(vocab_size, 768, 8, 8000, 8, 1024, False, 8, device=device)
    model = Transformer(vocab_size, 768, 8, 2048, 18, 1024, False, 8, device=device)
    # model = DataParallel(model)
    model.load_state_dict(torch.load('/root/output_test_new/epoch_50/pytorch_model.bin', map_location=device))
    model.eval()
    tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
    
    '''
    # num_gpus = torch.cuda.device_count()
    # captions_per_gpu = len(captions) // num_gpus
    # processes = []
    # for i in range(num_gpus):
    #     start_idx = i * captions_per_gpu
    #     end_idx = (i + 1) * captions_per_gpu if i != num_gpus - 1 else len(captions)
    #     p = mp.Process(target=process_caption, args=(i, captions[start_idx:end_idx], model, tokenizer, r_tokenizer))
    #     p.start()
    #     processes.append(p)
    # for p in processes:
    #     p.join()
    '''
    # src = "This short electronic song in C minor features a brass section, string ensemble, tenor saxophone, clean electric guitar, and slap bass, creating a melodic and slightly dark atmosphere. With a tempo of 124 BPM (Allegro) and a 4/4 time signature, the track incorporates a chord progression of C7/E, Eb6, and Bbm6, adding a touch of corporate and motivational vibes to the overall composition."
    src="This motivational electronic and pop song features a clean electric guitar, rock organ, synth voice, acoustic guitar, and vibraphone, creating a melodic and uplifting atmosphere. Set in the key of G# minor with a 4/4 time signature, the track moves at an energetic Allegro tempo of 120 beats per minute. The chord progression of Bbm7 and F# adds to the song's inspiring and corporate feel."
    # src = "Played at 149 beats per minute in 2/4 time signature and the key of G major, classical piece with instruments: bassoon, clarinet, flute, horn, oboe, and trumpet."
    # src= 'Played at 114 beats per minute in 1/4 time signature and the key of g# minor, classical piece with the following instruments: clarinet, english horn, flute, horn, piccolo, trombone, and trumpet.'
    inputs = tokenizer(src, return_tensors='pt', padding=True, truncation=True)
    input_ids = nn.utils.rnn.pad_sequence(inputs.input_ids, batch_first=True, padding_value=0)
    input_ids = input_ids.to(device)
    attention_mask =nn.utils.rnn.pad_sequence(inputs.attention_mask, batch_first=True, padding_value=0) 
    attention_mask = attention_mask.to(device)
    output = model.generate(input_ids, attention_mask,max_len=5000,temperature = 0.9)
    output_list = output[0].tolist()
    generated_midi = r_tokenizer.decode(output_list)
    generated_midi.dump_midi(f"../../output_e3_epoch_50_new.mid")
    

### Our Model

In [7]:
from transformers import T5Tokenizer, T5EncoderModel

class Transformer(Module):
    r"""A transformer model.
    The architecture is based on the paper "Attention Is All You Need".
    This code is adapted from https://huggingface.co/amaai-lab/text2midi/blob/main/transformer_model.py.
    Args:
        d_model: the number of expected features in the encoder/decoder inputs (default=512).
        nhead: the number of heads in the multiheadattention models (default=8).
        num_encoder_layers: the number of sub-encoder-layers in the encoder (default=6).
        num_decoder_layers: the number of sub-decoder-layers in the decoder (default=6).
        dim_feedforward: the dimension of the feedforward network model (default=2048).
        dropout: the dropout value (default=0.1).
        activation: the activation function of encoder/decoder intermediate layer, can be a string
            ("relu" or "gelu") or a unary callable. Default: relu
        custom_encoder: custom encoder (default=None).
        custom_decoder: custom decoder (default=None).
        layer_norm_eps: the eps value in layer normalization components (default=1e-5).
        batch_first: If ``True``, then the input and output tensors are provided
            as (batch, seq, feature). Default: ``False`` (seq, batch, feature).
        norm_first: if ``True``, encoder and decoder layers will perform LayerNorms before
            other attention and feedforward operations, otherwise after. Default: ``False`` (after).
        bias: If set to ``False``, ``Linear`` and ``LayerNorm`` layers will not learn an additive
            bias. Default: ``True``.
    """

    def __init__(self, n_vocab: int = 30000, d_model: int = 512, nhead: int = 8, max_len: int = 5000,
                 num_decoder_layers: int = 6, dim_feedforward: int = 2048, dropout: float = 0.1, 
                 activation: Union[str, Callable[[Tensor], Tensor]] = F.relu,
                 layer_norm_eps: float = 1e-5, batch_first: bool = True, norm_first: bool = False,
                 bias: bool = True, device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()
        torch._C._log_api_usage_once(f"torch.nn.modules.{self.__class__.__name__}")

        self.input_emb = nn.Embedding(n_vocab, d_model, **factory_kwargs)
        self.pos_encoder = PositionalEncoding(d_model, dropout, max_len).to(device)

        # Load the FLAN-T5 encoder
        self.encoder = T5EncoderModel.from_pretrained("google/flan-t5-base").to(device)
        # Freeze the encoder
        for param in self.encoder.parameters():
            param.requires_grad = False

        decoder_layer = TransformerDecoderLayer(d_model, nhead, dim_feedforward, dropout,
                                                activation, layer_norm_eps, batch_first, norm_first,
                                                bias, **factory_kwargs)
        decoder_norm = LayerNorm(d_model, eps=layer_norm_eps, bias=bias, **factory_kwargs)
        self.decoder = TransformerDecoder(decoder_layer, num_decoder_layers, decoder_norm)

        self.projection = nn.Linear(d_model, n_vocab).to(device)

        self._reset_parameters()

        self.d_model = d_model
        self.nhead = nhead

        self.batch_first = batch_first

    def forward(self, src: Tensor, src_mask: Tensor, tgt: Tensor, memory_mask: Optional[Tensor] = None,
                memory_key_padding_mask: Optional[Tensor] = None, tgt_is_causal: bool = True,
                memory_is_causal: bool = False) -> Tensor:
        r"""Take in and process masked source/target sequences.
        .. note::
            If a boolean tensor is provided for any of the [src/tgt/memory]_mask arguments, positions with a ``True`` value are
            not allowed to participate in the attention,
            which is the opposite of the definition for :attr:`attn_mask`
            in :func:`torch.nn.functional.scaled_dot_product_attention`.
        Args:
            src: the sequence to the encoder (required).
            src_attn_mask: the attention mask for the src sequence (required).
            tgt: the sequence to the decoder (required).
            tgt_mask: the additive mask for the tgt sequence (optional).
            memory_mask: the additive mask for the encoder output (optional).
            tgt_key_padding_mask: the Tensor mask for tgt keys per batch (optional).
            memory_key_padding_mask: the Tensor mask for memory keys per batch (optional).
            tgt_is_causal: If specified, applies a causal mask as ``tgt_mask``.
                Default: ``None``; try to detect a causal mask.
                Warning:
                ``tgt_is_causal`` provides a hint that ``tgt_mask`` is
                the causal mask. Providing incorrect hints can result in
                incorrect execution, including forward and backward
                compatibility.
            memory_is_causal: If specified, applies a causal mask as
                ``memory_mask``.
                Default: ``False``.
                Warning:
                ``memory_is_causal`` provides a hint that
                ``memory_mask`` is the causal mask. Providing incorrect
                hints can result in incorrect execution, including
                forward and backward compatibility.
        Shape:
            - src: :math:`(S, S)` for unbatched input, :math:`(S, N)` if `batch_first=False` or
              `(N, S)` if `batch_first=True`.
            - src_mask: :math:`(S, S)` or :math:`(N\cdot\text{num\_heads}, S, S)`.
            - tgt: :math:`(T, E)` for unbatched input, :math:`(T, N, E)` if `batch_first=False` or
              `(N, T, E)` if `batch_first=True`.
            - tgt_mask: :math:`(T, T)` or :math:`(N\cdot\text{num\_heads}, T, T)`.
            - memory_mask: :math:`(T, S)`.
            - src_key_padding_mask: :math:`(S)` for unbatched input otherwise :math:`(N, S)`.
            - tgt_key_padding_mask: :math:`(T)` for unbatched input otherwise :math:`(N, T)`.
            - memory_key_padding_mask: :math:`(S)` for unbatched input otherwise :math:`(N, S)`.
            Note: [src/tgt/memory]_mask ensures that position :math:`i` is allowed to attend the unmasked
            positions. If a BoolTensor is provided, positions with ``True``
            are not allowed to attend while ``False`` values will be unchanged. If a FloatTensor
            is provided, it will be added to the attention weight.
            [src/tgt/memory]_key_padding_mask provides specified elements in the key to be ignored by
            the attention. If a BoolTensor is provided, the positions with the
            value of ``True`` will be ignored while the position with the value of ``False`` will be unchanged.
            - output: :math:`(T, E)` for unbatched input, :math:`(T, N, E)` if `batch_first=False` or
              `(N, T, E)` if `batch_first=True`.
            Note: Due to the multi-head attention architecture in the transformer model,
            the output sequence length of a transformer is same as the input sequence
            (i.e. target) length of the decoder.
            where :math:`S` is the source sequence length, :math:`T` is the target sequence length, :math:`N` is the
            batch size, :math:`E` is the feature number
        """
        if src.dim() != tgt.dim():
            raise RuntimeError("the number of dimensions in src and tgt must be equal")

        memory = self.encoder(src, attention_mask=src_mask).last_hidden_state

        tgt = self.input_emb(tgt) * math.sqrt(self.d_model)
        tgt = self.pos_encoder(tgt)

        output = self.decoder(tgt, memory, memory_mask=memory_mask,                                
                            memory_key_padding_mask=memory_key_padding_mask,
                            tgt_is_causal=tgt_is_causal, memory_is_causal=memory_is_causal)
        
        output = self.projection(output)

        return output
        
    def generate(self, src: Tensor, src_mask: Tensor, max_len: int = 100, temperature: float = 1.0):
        r"""Generate a sequence of tokens from the given inputs.
        Args:
            src: the sequence to the encoder (required).
            src_mask: the attention mask for the src sequence (required).
            max_len: the maximum length of the sequence to generate (default=100).
            temperature: the temperature for the softmax (default=1.0).
        Returns:
            torch.Tensor: The generated sequence of tokens.
        """
        if src.dim() != 2:
            raise RuntimeError("The src tensor should be 2-dimensional")
        tgt_fin = torch.full((src.size(0), 1), 1, dtype=torch.long, device=src.device)
        # values = [21631, 8, 10, 9, 6, 7, 17, 21632, 11474, 20626, 21151, 9426, 20627, 21143, 11476, 20640, 21143, 11477, 20655, 21145, 11476, 20669, 21145, 11477, 20683, 21145, 13527, 20697, 21146, 13529, 20712, 21145, 7013, 20769, 21143, 7006, 20769, 21143, 7006, 20769, 21141, 7009, 20769, 21143, 9426, 20797, 21144, 11474, 20797, 21173, 11476, 20812, 21144, 11477, 20826, 21145, 11476, 20840, 21145, 11477, 20855, 21145, 13527, 20869, 21144, 13529, 20883, 21143, 7006, 20940, 21139, 7013, 20940, 21140, 7006, 20940, 21147, 7009, 20940, 21147, 11474, 20969, 21144, 11474, 20969, 21170, 11476, 20983, 21144, 11477, 20997, 21145, 11476, 21012, 21144, 11477, 21026, 21144, 11479, 21040]
        # values_tensor = torch.tensor(values, dtype=torch.long, device=src.device)
        # tgt_fin = values_tensor.unsqueeze(0).repeat(src.size(0), 1)
        for i in tqdm(range(max_len)):
            max_index = tgt_fin.max()
            # assert max_index < 21634, "tgt_fin contains index out of range. Adjust n_vocab or fix tgt_fin indices."
            tgt = tgt_fin
            output = self.forward(src, src_mask, tgt, memory_mask=None,                                
                                    memory_key_padding_mask=None,
                                    tgt_is_causal=True, memory_is_causal=False)          
            # logits = self.projection(output)
            logits = output
            output = F.log_softmax(logits/temperature, dim=-1)
            output = output.view(-1, output.size(-1))
            next_tokens = torch.multinomial(torch.exp(output), 1)[-1] # taking the last logit and adding to the sequence
            tgt_fin = torch.cat((tgt_fin, next_tokens.unsqueeze(-1)), dim=1)
        return tgt_fin[:, 1:]


    def _reset_parameters(self):
        r"""Initiate parameters in the transformer model."""
        for p in self.parameters():
            if p.dim() > 1:
                xavier_uniform_(p)

### Pretraining

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import T5EncoderModel




class Text2MidiModel(nn.Module):
    def __init__(self, remi_vocab_size, d_model=256, n_layers=4):
        super().__init__()

        # Use FLAN-T5 encoder
        self.encoder = T5EncoderModel.from_pretrained("google/flan-t5-small")

        # Tiny decoder
        self.decoder = TinyTransformerDecoder(
            vocab_size=remi_vocab_size,
            d_model=d_model,
            n_layers=n_layers,
            n_heads=4,
            dim_ff=512,
            max_len=2048
        )

    def forward(self, input_ids, attention_mask, decoder_input_ids, labels=None):
        # Encode text
        enc_outputs = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask
        ).last_hidden_state  # [B, T_enc, D]

        # Decode REMI tokens
        logits = self.decoder(
            decoder_input_ids,
            enc_outputs
        )  # [B, T_dec, vocab]

        loss = None
        if labels is not None:
            loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)),
                labels.view(-1),
                ignore_index=-100
            )

        return {"logits": logits, "loss": loss}


In [28]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from tqdm import tqdm
from torch.utils.data import DataLoader

# Config
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# DEVICE = "cpu"

EPOCHS = 10
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 0.01
GRAD_ACCUM_STEPS = 2
SAVE_EVERY = 1
OUTPUT_DIR = "./checkpoints"

REMI_VOCAB_SIZE = remi_tokenizer.vocab_size  # your REMI tokenizer
MAX_SEQ_LEN = 1024

# Decoder
class DecoderOnlyModel(nn.Module):
    def __init__(self, remi_vocab_size, d_model=256, n_layers=4):
        super().__init__()
        from copy import deepcopy

        self.decoder = TinyTransformerDecoder(
            vocab_size=remi_vocab_size,
            d_model=d_model,
            n_layers=n_layers,
            n_heads=4,
            dim_ff=512,
            max_len=MAX_SEQ_LEN
        )

    def forward(self, decoder_input_ids, labels=None):
        # dummy encoder outputs for compatibility
        B, T = decoder_input_ids.shape
        dummy_enc = torch.zeros(B, 1, self.decoder.layers[0].self_attn.embed_dim, device=decoder_input_ids.device)
        logits = self.decoder(decoder_input_ids, dummy_enc)

        loss = None
        if labels is not None:
            loss = F.cross_entropy(
                logits.reshape(-1, logits.size(-1)),
                labels.reshape(-1),
                ignore_index=-100
            )
        return {"logits": logits, "loss": loss}

# Model and optimizer
model = DecoderOnlyModel(REMI_VOCAB_SIZE).to(DEVICE)
model.load_state_dict(torch.load("./checkpoints/decoder_epoch_10.pt"))
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
losses = []

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Pre-training loop
def train_decoder_only(model, dataloader):
    model.train()
    global_step = 0

    for epoch in range(11, EPOCHS + 11):
        loop = tqdm(dataloader, desc=f"Epoch {epoch}/{EPOCHS}")
        optimizer.zero_grad()

        for step, batch in enumerate(loop):
            batch = batch.to(DEVICE)
            decoder_input_ids = batch[:, :-1]
            labels = batch[:, 1:]

            # --- forward & loss ---
            out = model(decoder_input_ids, labels)
            loss = out["loss"] / GRAD_ACCUM_STEPS

            # --- backward ---
            loss.backward()

            if (step + 1) % GRAD_ACCUM_STEPS == 0:
                optimizer.step()
                optimizer.zero_grad()
                global_step += 1

            loop.set_postfix(loss=loss.item() * GRAD_ACCUM_STEPS)

        # --- save checkpoint ---
        if epoch % SAVE_EVERY == 0:
            ckpt_path = os.path.join(OUTPUT_DIR, f"decoder_epoch_{epoch}.pt")
            torch.save(model.state_dict(), ckpt_path)
            print(f"Saved checkpoint: {ckpt_path}")
            losses.append(loss)
            print(loss)
    print("Losses:", losses)
            

# Initialize dataset and dataloader
# gigamidi2_path = "Text2midi/gigamidi2_100k"
# gm_ds = MidiOnlyDataset(gigamidi2_path, remi_tokenizer=remi_tokenizer, max_len=MAX_SEQ_LEN)
# loader = DataLoader(
#     gm_ds,
#     batch_size=32,
#     shuffle=True,
# )

# train_decoder_only(model, loader)


/tmp/ipykernel_2613/1048164564.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("./checkpoints/decoder_epoch_10.pt"))


MIDI files found: 100000


Epoch 11/10:  65%|██████▌   | 2044/3125 [36:54<19:55,  1.11s/it, loss=0.608]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Epoch 12/10: 100%|██████████| 3125/3125 [55:35<00:00,  1.07s/it, loss=0.725]  


Saved checkpoint: ./checkpoints/decoder_epoch_12.pt
tensor(0.3626, device='cuda:0', grad_fn=<DivBackward0>)


Epoch 13/10: 100%|██████████| 3125/3125 [56:02<00:00,  1.08s/it, loss=0.608]  


...

Saved checkpoint: ./checkpoints/decoder_epoch_14.pt
tensor(0.3130, device='cuda:0', grad_fn=<DivBackward0>)


Epoch 15/10: 100%|██████████| 3125/3125 [55:14<00:00,  1.06s/it, loss=0.663]  


Saved checkpoint: ./checkpoints/decoder_epoch_15.pt
tensor(0.3314, device='cuda:0', grad_fn=<DivBackward0>)


Epoch 16/10:   1%|          | 38/3125 [00:38<52:36,  1.02s/it, loss=0.53]   


KeyboardInterrupt: 

### Pretokenize MIDI Files

In [13]:
from datasets import load_dataset

midicaps = load_dataset("amaai-lab/MidiCaps", split="train")
midicaps

Dataset({
    features: ['location', 'caption', 'genre', 'genre_prob', 'mood', 'mood_prob', 'key', 'time_signature', 'tempo', 'tempo_word', 'duration', 'duration_word', 'chord_summary', 'chord_summary_occurence', 'instrument_summary', 'instrument_numbers_sorted', 'all_chords', 'all_chords_timestamps', 'test_set'],
    num_rows: 168385
})

In [14]:
import random

num_samples = 100

# Generate random indices
random_indices = random.sample(range(len(midicaps)), num_samples)

# Select the random samples
sample = midicaps.select(random_indices)

print(sample)

Dataset({
    features: ['location', 'caption', 'genre', 'genre_prob', 'mood', 'mood_prob', 'key', 'time_signature', 'tempo', 'tempo_word', 'duration', 'duration_word', 'chord_summary', 'chord_summary_occurence', 'instrument_summary', 'instrument_numbers_sorted', 'all_chords', 'all_chords_timestamps', 'test_set'],
    num_rows: 100
})


In [11]:
import os
import torch
from tqdm import tqdm
from pathlib import Path
from transformers import T5Tokenizer
from multiprocessing import Pool

def process_item(args):
    (
        item,
        midi_folder,
        out_folder,
        remi_tokenizer,
        t5_tokenizer,
        max_decoder_len,
        max_text_len
    ) = args

    caption = item["caption"]
    midi_loc = item["location"]

    midi_path = os.path.join(midi_folder, midi_loc)

    # Output path (same name but .pt)
    base = os.path.splitext(midi_loc)[0]
    out_path = os.path.join(out_folder, base + ".pt")

    # Skip if already pretokenized
    if os.path.exists(out_path):
        return None

    # Create subdirectories (mirrored structure)
    os.makedirs(os.path.dirname(out_path), exist_ok=True)

    # ---- MIDI → REMI ----
    tokens = remi_tokenizer(midi_path)
    if len(tokens.ids) == 0:
        midi = [
            remi_tokenizer["BOS_None"],
            remi_tokenizer["EOS_None"],
        ]
    else:
        midi = (
            [remi_tokenizer["BOS_None"]]
            + tokens.ids
            + [remi_tokenizer["EOS_None"]]
        )

    # Pad/truncate
    if len(midi) < max_decoder_len:
        labels = torch.nn.functional.pad(
            torch.tensor(midi),
            (0, max_decoder_len - len(midi))
        )
    else:
        labels = torch.tensor(midi[:max_decoder_len])

    labels = labels.long()

    # ---- Caption → T5 ----
    enc = t5_tokenizer(
        caption,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=max_text_len,
    )

    input_ids = enc["input_ids"][0]
    attention_mask = enc["attention_mask"][0]

    # ---- Save ----
    torch.save(
        {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        },
        out_path
    )

    return None


def run_pretokenization(
    hf_items,
    midi_folder,
    out_folder,
    remi_tokenizer,
    max_decoder_len=1024,
    max_text_len=256,
    num_workers=8
):
    os.makedirs(out_folder, exist_ok=True)

    t5_tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")

    # Prepare arguments for each process
    args_list = [
        (
            item,
            midi_folder,
            out_folder,
            remi_tokenizer,
            t5_tokenizer,
            max_decoder_len,
            max_text_len
        )
        for item in hf_items
    ]

    with Pool(num_workers) as p:
        list(
            tqdm(
                p.imap_unordered(process_item, args_list, chunksize=8),
                total=len(args_list)
            )
        )

    print("Pretokenization complete.")

In [25]:
from datasets import load_dataset

run_pretokenization(
    hf_items=midicaps,
    midi_folder="midicaps",
    out_folder="midicaps_pretokenized",
    remi_tokenizer=remi_tokenizer,
    max_decoder_len=1024,
    max_text_len=256,
    num_workers=8
)

100%|██████████| 168385/168385 [28:16<00:00, 99.24it/s] 


Pretokenization complete.


### Training Dataset (Pretokenized) (MIDICaps)

In [26]:
import torch

class TextMidiDataset(torch.utils.data.Dataset):
    """
    Returns:
        input_ids:      [1, max_text_toks]
        attention_mask: [1, max_text_toks]
        labels:         [decoder_max_len]
    """

    def __init__(self, hf_items, pretok_folder,
                 max_decoder_len=1024, max_text_toks=256):
        self.hf_items = hf_items
        self.pretok_folder = pretok_folder
        self.decoder_max_len = max_decoder_len
        self.max_text_toks = max_text_toks

        print("Length of dataset:", len(self.hf_items))

    def __len__(self):
        return len(self.hf_items)

    def __getitem__(self, idx):
        item = self.hf_items[idx]
        midi_loc = item["location"]

        # Build path to pretokenized .pt file
        base = os.path.splitext(midi_loc)[0]
        pt_path = os.path.join(self.pretok_folder, base + ".pt")

        # Load pretokenized sample
        sample = torch.load(pt_path, weights_only="False")

        input_ids = sample["input_ids"].unsqueeze(0)        # [1, max_text_toks]
        attention_mask = sample["attention_mask"].unsqueeze(0)  # [1, max_text_toks]
        labels = sample["labels"]                           # [decoder_max_len]

        return input_ids, attention_mask, labels


In [29]:
pretok_folder = "midicaps_pretokenized"
ds = TextMidiDataset(midicaps, pretok_folder)
print(ds[0][0].shape, ds[0][1].shape, ds[0][2].shape)

Length of dataset: 168385
torch.Size([1, 256]) torch.Size([1, 256]) torch.Size([1024])


### Training Dataset (Legacy) (MIDICaps)

In [9]:
import os
import random
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.nn import functional as F
from transformers import T5Tokenizer
from spacy.lang.en import English

# class TextMidiDataset(Dataset):
#     """
#     Returns:
#         input_ids:      [1, L_text]
#         attention_mask: [1, L_text]
#         labels:         [decoder_max_len]
#     """

#     def __init__(self, midi_folder, hf_items, remi_tokenizer,
#                  max_decoder_len=1024, max_text_toks=256):
#         self.hf_items = hf_items

#         self.midi_folder = midi_folder
#         self.remi_tokenizer = remi_tokenizer

#         self.decoder_max_len = max_decoder_len
#         self.max_text_toks = max_text_toks

#         self.t5_tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")

#         self.nlp = English()
#         self.nlp.add_pipe("sentencizer")

#         print("Length of dataset:", len(self.hf_items))

#     def __len__(self):
#         return len(self.hf_items)

#     def drop_sentences(self, caption: str):
#         """Drop 20–50% of half of the sentences to avoid overfitting"""
#         if random.random() < 0.5:
#             return caption

#         sentences = list(self.nlp(caption).sents)
#         n = len(sentences)
#         if n == 0:
#             return caption

#         frac = (20 + random.random() * 30) / 100.0
#         k = int(np.ceil(frac * n))

#         drop_idx = set(np.random.choice(n, k, replace=False))
#         kept = [sentences[i].text for i in range(n) if i not in drop_idx]
#         return " ".join(kept)

#     def __getitem__(self, idx):
#         item = self.hf_items[idx]
#         caption = item["caption"]
#         midi_loc = item["location"]

#         midi_path = os.path.join(self.midi_folder, midi_loc)

#         # Convert MIDI to REMI tokens
#         tokens = self.remi_tokenizer(midi_path)
#         if len(tokens.ids) == 0:
#             midi = [self.remi_tokenizer["BOS_None"], self.remi_tokenizer["EOS_None"]]
#         else:
#             midi = [self.remi_tokenizer["BOS_None"]] + tokens.ids + [self.remi_tokenizer["EOS_None"]]

#         if len(midi) < self.decoder_max_len:
#             labels = F.pad(torch.tensor(midi), (0, self.decoder_max_len - len(midi)))
#         else:
#             labels = torch.tensor(midi[:self.decoder_max_len])

#         labels = labels.long()

#         # Caption to T5 encoder tokens
#         caption = self.drop_sentences(caption)
#         enc = self.t5_tokenizer(
#             caption,
#             return_tensors="pt",
#             padding="max_length",
#             truncation=True,
#             max_length=self.max_text_toks,
#         )

#         return (
#             enc["input_ids"],      # [1, max_text_toks]
#             enc["attention_mask"], # [1, max_text_toks]
#             labels                 # [decoder_max_len]
#         )

### Training

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.optim.lr_scheduler import CosineAnnealingLR
import csv

# Config
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

EPOCHS = 15
LEARNING_RATE = 1e-5
WEIGHT_DECAY = 0.01
GRAD_ACCUM_STEPS = 2
SAVE_EVERY = 1
OUTPUT_DIR = "./checkpoints_full_model"
LOSS_LOG_FILE = os.path.join(OUTPUT_DIR, "training_loss.csv")

MAX_TEXT_LEN = 256
MAX_REMI_LEN = 1024

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# FULL MODEL: Frozen FLAN-T5 Encoder + Pre-trained Decoder
class FullSeq2Seq(nn.Module):
    def __init__(self, remi_vocab_size, d_model=256, n_layers=4):
        super().__init__()
        from transformers import T5EncoderModel
        self.encoder = T5EncoderModel.from_pretrained("google/flan-t5-base")
        for p in self.encoder.parameters():
            p.requires_grad = False
        enc_hidden = self.encoder.config.d_model

        self.decoder = TinyTransformerDecoder(
            vocab_size=remi_vocab_size,
            d_model=d_model,
            n_layers=n_layers,
            n_heads=4,
            dim_ff=512,
            max_len=MAX_REMI_LEN
        )

        self.enc_proj = nn.Linear(enc_hidden, d_model)

    def forward(self, text_input_ids, attention_mask, decoder_input_ids, labels=None):
        with torch.no_grad():
            enc_out = self.encoder(input_ids=text_input_ids, attention_mask=attention_mask).last_hidden_state
        enc_out = self.enc_proj(enc_out)
        logits = self.decoder(decoder_input_ids, enc_out)

        loss = None
        if labels is not None:
            loss = F.cross_entropy(
                logits.reshape(-1, logits.size(-1)),
                labels.reshape(-1),
                ignore_index=-100
            )
        return {"logits": logits, "loss": loss}

# Training loop
def train_full(model, dataloader, optimizer, scheduler=None):
    model.train()
    for epoch in range(15, EPOCHS + 15):
        loop = tqdm(dataloader, desc=f"Epoch {epoch}/{EPOCHS}")
        optimizer.zero_grad()
        running_loss = 0.0
        step_count = 0

        for step, batch in enumerate(loop):
            text_ids, attn_mask, remi_ids = batch
            text_ids = text_ids.squeeze(1).to(DEVICE)
            attn_mask = attn_mask.squeeze(1).to(DEVICE)
            remi_ids = remi_ids.to(DEVICE)

            dec_in = remi_ids[:, :-1]
            labels = remi_ids[:, 1:]

            out = model(text_ids, attn_mask, dec_in, labels)
            loss = out["loss"] / GRAD_ACCUM_STEPS
            loss.backward()
            running_loss += loss.item() * GRAD_ACCUM_STEPS
            step_count += 1

            if (step + 1) % GRAD_ACCUM_STEPS == 0:
                optimizer.step()
                if scheduler is not None:
                    scheduler.step()
                optimizer.zero_grad()

            loop.set_postfix(loss=loss.item() * GRAD_ACCUM_STEPS)

        # Average epoch loss
        avg_loss = running_loss / step_count
        print(f"Epoch {epoch} finished. Avg loss = {avg_loss:.4f}")

        # Append to CSV
        with open(LOSS_LOG_FILE, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([epoch, avg_loss])

        # Save checkpoint
        if epoch % SAVE_EVERY == 0:
            ckpt_path = os.path.join(OUTPUT_DIR, f"full_model_epoch_{epoch}.pt")
            torch.save(model.state_dict(), ckpt_path)
            print(f"Saved checkpoint → {ckpt_path}")


# Initialize dataset and dataloader
# midicaps_path = "./midicaps"
# mc_ds = TextMidiDataset(midicaps_path, 
#                         midicaps, 
#                         remi_tokenizer=remi_tokenizer, 
#                         max_decoder_len=MAX_REMI_LEN, 
#                         max_text_toks=MAX_TEXT_LEN)
pretok_folder = "midicaps_pretokenized"
mc_ds = TextMidiDataset(midicaps, 
                        pretok_folder, 
                        max_decoder_len=MAX_REMI_LEN, 
                        max_text_toks=MAX_TEXT_LEN)
dataloader = DataLoader(
    mc_ds,
    batch_size=32,
    shuffle=True,
)

# Initialize model, with pretrained decoder
model = FullSeq2Seq(remi_tokenizer.vocab_size).to(DEVICE)

checkpoint = "./checkpoints_full_model/full_model_epoch_14.pt"
ckpt_state = torch.load(checkpoint, weights_only="False", map_location="cuda")

# remove "decoder." prefix
new_ckpt = {}
for k, v in ckpt_state.items():
    if k.startswith("decoder."):
        new_ckpt[k[len("decoder."):]] = v
    else: new_ckpt[k] = v # for safety 

model.decoder.load_state_dict(new_ckpt, strict=False)

# Initialize optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS * len(dataloader))

model.to(DEVICE)
train_full(model, dataloader, optimizer, scheduler)

Length of dataset: 168385


Epoch 15/15:   2%|▏         | 106/5263 [01:54<1:32:54,  1.08s/it, loss=0.948]

In [ ]:
def count_parameters(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total, trainable

total, trainable = count_parameters(model)

print(f"Total parameters: {total:,}")
print(f"Trainable parameters: {trainable:,}")
print(f"Frozen parameters: {total - trainable:,}")

In [55]:
import gc

# del loader
# del gm_ds
del model
del dataloader
del mc_ds
gc.collect()
torch.cuda.empty_cache()

In [56]:
!nvidia-smi

Sat Dec  6 02:24:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.195.03             Driver Version: 570.195.03     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX 2000 Ada Gene...    On  |   00000000:81:00.0 Off |                  Off |
| 31%   39C    P2             23W /   70W |    1954MiB /  16380MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Training (MidiCaps)

In [ ]:
import os
import random
import torch
from torch.utils.data import Dataset
from torch.nn import functional as F
from transformers import T5Tokenizer
from spacy.lang.en import English
import numpy as np

class MidiCaptionDataset(Dataset):
    """
    Supervised training dataset: each item has a caption and a MIDI file path.
    """
    def __init__(self, captions_list, midi_folder, remi_tokenizer, max_seq_len, shuffle=True, drop_sentences=True):
        """
        captions_list: list of dicts with keys ['caption', 'location']
        midi_folder: folder containing MIDI files
        remi_tokenizer: pre-trained REMI tokenizer
        max_seq_len: maximum decoder sequence length
        """
        self.captions = captions_list
        if shuffle:
            random.shuffle(self.captions)

        self.midi_folder = midi_folder
        self.remi_tokenizer = remi_tokenizer
        self.max_seq_len = max_seq_len
        self.drop_sentences = drop_sentences

        # Load sentencizer
        self.nlp = English()
        self.nlp.add_pipe('sentencizer')

        # Load T5 tokenizer
        self.t5_tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")

    def __len__(self):
        return len(self.captions)

    def __getitem__(self, idx):
        item = self.captions[idx]
        caption = item['caption']
        midi_path = os.path.join(self.midi_folder, item['location'])

        # Tokenize MIDI
        tokens = self.remi_tokenizer(midi_path)
        if len(tokens.ids) == 0:
            tokenized_midi = [self.remi_tokenizer["BOS_None"], self.remi_tokenizer["EOS_None"]]
        else:
            tokenized_midi = [self.remi_tokenizer["BOS_None"]] + tokens.ids + [self.remi_tokenizer["EOS_None"]]

        # Optionally drop sentences for data augmentation
        if self.drop_sentences and random.random() > 0.5:
            sentences = list(self.nlp(caption).sents)
            n = len(sentences)
            if n < 4:
                drop_n = int(np.floor((20 + random.random()*30)/100 * n))
            else:
                drop_n = int(np.ceil((20 + random.random()*30)/100 * n))
            drop_idx = np.random.choice(n, drop_n, replace=False)
            new_sentences = [sentences[i] for i in range(n) if i not in drop_idx.tolist()]
            caption = " ".join([s.text for s in new_sentences])

        # Tokenize caption
        text_inputs = self.t5_tokenizer(caption, return_tensors='pt', padding=True, truncation=True)
        input_ids = text_inputs['input_ids']
        attention_mask = text_inputs['attention_mask']

        # Pad/truncate MIDI sequence
        if len(tokenized_midi) < self.max_seq_len:
            labels = F.pad(torch.tensor(tokenized_midi),
                           (0, self.max_seq_len - len(tokenized_midi))).to(torch.int64)
        else:
            labels = torch.tensor(tokenized_midi[:self.max_seq_len]).to(torch.int64)

        return input_ids, attention_mask, labels


In [51]:
!python -c "import torch; import torchvision; print(torch.__version__, torchvision.__version__)"
!pip uninstall torchvision -y
!pip install torch==2.5.1 torchvision==0.20.1 --index-url https://download.pytorch.org/whl/cu124

2.8.0+cu128 0.23.0+cu128
Found existing installation: torchvision 0.23.0+cu128
Uninstalling torchvision-0.23.0+cu128:
  Successfully uninstalled torchvision-0.23.0+cu128
Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.2/908.2 MB 72.2 MB/s  0:00:09:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 67.0 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.1 MB/s  0:00:00 eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.2 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.3 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 71.3 MB/s  0:00:07:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 87.7 MB/s  0:00:03:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 91.7 MB/s  0:00:02:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 80.9 MB/s  0:00:00m0:00: